In [17]:
import json
import http.client
import pandas as pd
# from dotenv import load_dotenv
import os
# load_dotenv()  # take environment variables from .env.

In [18]:
def get_token(access_key): 
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ''
    headers = {
    'AccessKey': access_key
    }
    conn.request("GET", "/uraDataService/insertNewToken.action", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))['Result']

In [19]:
def get_result(token, access_key, route):
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ""
    headers = {
        "AccessKey": access_key,
        "Token": token,
        "User-Agent": "PostmanRuntime/7.26.8",
    }
    conn.request("GET", route, payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))['Result']


def get_all_results(token, access_key, routes):
    result = []
    for route in routes:
        result.extend(get_result(token, access_key, route))
    return pd.DataFrame(result)


In [21]:

def get_all_ura():
    private_transactions_routes = [
        "/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=" + str(i)
        for i in range(1, 5)
    ]
    private_rental_routes = [
        f"/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod={q}"
        # start from 14q1 to 23q1
        for q in [f"{y}q{i}" for y in range(14, 24) for i in range(1, 5)]
    ]  

    planning_decisions_routes = [
        "/uraDataService/invokeUraDS?service=Planning_Decision&year=" + str(i)
        for i in range(2000, 2024)
    ]

    access_key = "480de617-6aee-4c71-a04a-3b6c3a9596b2"
    token = get_token(access_key)

    # get_all_results(token, access_key, private_transactions_routes).to_csv(
    #     "../data/private_transactions.csv", index=False
    # )
    # get_all_results(token, access_key, private_rental_routes).to_csv(
    #     "../data/private_rental.csv", index=False
    # )
    # get_all_results(token, access_key, planning_decisions_routes).to_csv(
    #     "../data/planning_decisions.csv", index=False
    # )

    df_private_transactions = get_all_results(token, access_key, private_transactions_routes)

    df_private_rental = get_all_results(token, access_key, private_rental_routes)

    df_planning_decisions = get_all_results(token, access_key, planning_decisions_routes)

    return df_private_transactions, df_private_rental, df_planning_decisions


In [22]:
df_private_transactions, df_private_rental, df_planning_decisions = get_all_ura()

In [23]:
df_private_transactions

,street,x,project,y,transaction,marketSegment
0,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,"[{'area': '524.3', 'floorRange': '-', 'noOfUni...",RCR
1,NEO PEE TECK LANE,20914.1988965525,LANDED HOUSING DEVELOPMENT,30589.1070785135,"[{'area': '159.3', 'floorRange': '-', 'noOfUni...",RCR
2,COVE DRIVE,28382.47067,TURQUOISE,25008.33592,"[{'area': '224', 'floorRange': '01-05', 'noOfU...",CCR
3,BUKIT TERESA ROAD,27054.52601,LANDED HOUSING DEVELOPMENT,28712.63181,"[{'area': '435.2', 'floorRange': '-', 'noOfUni...",RCR
4,ALEXANDRA VIEW,26204.50079,ARTRA,30270.56219,"[{'area': '77', 'floorRange': '06-10', 'noOfUn...",RCR
...,...,...,...,...,...,...
4015,SELETAR GREEN WALK,32600.3346640566,LUXUS HILLS,40227.8621375907,"[{'area': '241.4', 'floorRange': '-', 'noOfUni...",OCR
4016,DEDAP LINK,31754.0572171626,DEDAP GARDENS,41051.6031486072,"[{'area': '423.4', 'floorRange': '-', 'noOfUni...",OCR
4017,YISHUN CENTRAL 1,28390.14699,NORTH PARK RESIDENCES,45560.43713,"[{'area': '81', 'floorRange': '01-05', 'noOfUn...",OCR
4018,GERALD CRESCENT,32836.6471,GERALD MUGLISTON ESTATE,40981.74326,"[{'area': '660.7', 'floorRange': '-', 'noOfUni...",OCR


In [24]:
df_private_rental


,street,x,project,y,rental
0,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,"[{'areaSqm': '190-200', 'leaseDate': '0318', '..."
1,HILLVIEW AVENUE,19806.57039,MERAWOODS,37591.36252,"[{'areaSqm': '100-110', 'leaseDate': '0318', '..."
2,BAYSHORE ROAD,39752.4696,NON-LANDED HOUSING DEVELOPMENT,32815.94888,"[{'areaSqm': '80-90', 'leaseDate': '0318', 'pr..."
3,ROSE LANE,34610.01791,NON-LANDED HOUSING DEVELOPMENT,32565.15659,"[{'areaSqm': '140-150', 'leaseDate': '0318', '..."
4,LEONIE HILL,27971.56162,RIVERSHIRE,31006.7352,"[{'areaSqm': '100-110', 'leaseDate': '0318', '..."
...,...,...,...,...,...
56170,HILLVIEW AVENUE,20431.67213,GLENDALE PARK,38295.46048,"[{'areaSqm': '110-120', 'leaseDate': '0123', '..."
56171,ENG KONG TERRACE,20774.10384,KISMIS RESIDENCES,35442.81759,"[{'areaSqm': '150-200', 'leaseDate': '0123', '..."
56172,SIXTH AVENUE,23507.96418,PALMS @ SIXTH AVENUE,34065.24232,"[{'areaSqm': '450-500', 'leaseDate': '0123', '..."
56173,PUNGGOL FIELD,35969.63449,PRIVE,42560.406,"[{'areaSqm': '70-80', 'leaseDate': '0123', 'pr..."


In [25]:
df_planning_decisions

,decision_date,address,submission_desc,decision_type,appl_type,mkts_lotno,dr_id,submission_no,decision_no
0,01/01/2000,ORCHARD ROAD,PROPOSED OUTDOOR REFRESHMENT AREA FOR UNIT #01-02,Written Permission,Addition/Alteration to Buildings (Others),"TS21 551, TS21 554PT, TS21 787, TS21 788, TS21...",55888b,071098-27Z3-Z000,P071098-27Z3-Z000
1,03/01/2000,CORONATION ROAD WEST,PROPOSED AMENDMENT TO APPROVED ERECTION OF 6 U...,Written Permission,Amendment to New Erection,MK04 110,55911,120496-34D2-A022,P120496-34D2-A022
2,03/01/2000,BEECHWOOD GROVE,PROPOSED AMENDMENT TO APPROVED ERECTION OF 2 U...,Written Permission,Amendment to New Erection,"MK13 2903, MK13 2904",53075,300196-24H3-A011,P300196-24H3-A011
3,03/01/2000,VALLEY ROAD,SUBDIVISION OF LAND INTO 2 PLOTS,Written Permission,Subdivision of Land,MK22 5115,46971b,071299-07I1-Z000,P071299-07I1-Z000
4,03/01/2000,554 SEMBAWANG ROAD,PROPOSED ADDITIONS AND ALTERATIONS TO THE EXIS...,Written Permission,Addition/Alteration to Landed Housing,MK19 673,47077b,021299-24D1-Z000,P021299-24D1-Z000
...,...,...,...,...,...,...,...,...,...
175845,17/03/2023,"42,41,40,39,38,37,36,35,34,33,32,31,30,29 MENG...",PROPOSED SUBDIVISION OF LAND ON LOT 551T MK13 ...,Written Permission,Subdivision of Land,MK13 00551T,284159,170223-12D1-Z000,P170223-12D1-Z000
175846,17/03/2023,1 ANDREW ROAD,PROPOSED NEW ERECTION OF A 2-STOREY ENVELOPE C...,Written Permission,New Erection,MK17 07664W,284164,050322-55G2-Z000,P050322-55G2-Z000
175847,17/03/2023,143 CECIL STREET,PROPOSED CHANGE OF USE OF UNIT #02-01 FROM SHO...,Written Permission,Change of Use,TS02 00257C,284182,070123-01D1-Z000,P070123-01D1-Z000
175848,17/03/2023,3 ANSON ROAD,PROPOSED CHANGE OF USE OF OFFICE UNIT #07-02 T...,Written Permission,Change of Use,"TS30 00145M,TS23 00234M,TS23 01175X",284176,151222-23J2-Z000,P151222-23J2-Z000
